# Hierarchical Indexing

3D and 4D data can be handled in Python via the inbuild `Panel` and `Panel4D` objects

Hierarchical indexing a more common approach to this using multiple index levels within a single index. 

Can be accessed using `Series` and `DataFrame` objects

In [1]:
import pandas as pd
import numpy as np

## A Multiple Indexed `Series`

2D data within a 1D object

The bad way:

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]

pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [4]:
for i in pop.index:
    print(i)

('California', 2000)
('California', 2010)
('New York', 2000)
('New York', 2010)
('Texas', 2000)
('Texas', 2010)


In [5]:
for i in pop.values:
    print(i)

33871648
37253956
18976457
19378102
20851820
25145561


How can we access all values from 2010?

In [6]:
# this is not nice

pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

## Pandas to the rescue...

Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas `MultiIndex` type gives us the type of operations we wish to have:

In [7]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [8]:
index.levels

FrozenList([['California', 'New York', 'Texas'], [2000, 2010]])

Notice that the `MultiIndex` contains multiple levels of indexing–in this case, the state names and the years

Re-indexing results in the following:

In [9]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Back to our original question...

In [10]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [11]:
pop['California', :]

2000    33871648
2010    37253956
dtype: int64

## `MultiIndex` as an extra dimension

The previous slides look familiar: why not just store this information with some extra indices and column labels?

In [12]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [13]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

So, why are we doing this? 

It gives us flexibility in types of data that we can represent.

Example: adding an additional column of demographic data for each state for each year:

In [14]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

We can use ufuncs on hierarchical indices too:

In [15]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


This allows us to easily and quickly manipulate and explore even high-dimensional data.

## Methods of MultiIndex Creation

Simply pass a list of two or more index arrays to the constructor of a `Series` or `DataFrame` object:

In [16]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.361116  0.258876
  2  0.164748  0.364393
b 1  0.024997  0.533592
  2  0.041892  0.306843

In [17]:
df.unstack()

data1               data2          
          1         2         1         2
a  0.361116  0.164748  0.258876  0.364393
b  0.024997  0.041892  0.533592  0.306843

You can pass a dictionary with tuples as keys:

In [18]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

We can also give the levels of the MultiIndex names to keep track of the meaning of indices:

In [19]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## MultiIndex for Columns

In a `DataFrame`, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well. 

In [20]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      19.0  37.6  58.0  38.1  52.0  36.5
     2      37.0  37.3  33.0  36.7  46.0  37.8
2014 1      35.0  37.4  30.0  36.2  39.0  38.2
     2      45.0  35.6  35.0  38.3  37.0  36.2

This is where multi-indexing comes in handy. With this in place, we can now observe the records for a given subject over a given time-period:

In [21]:
health_data['Bob']

type          HR  Temp
year visit            
2013 1      19.0  37.6
     2      37.0  37.3
2014 1      35.0  37.4
     2      45.0  35.6

## How can we index and slice a MultiIndex?

Let's look at this for a `Series` first:

In [22]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [23]:
pop['California', 2000]

33871648

In [24]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [25]:
pop[[('California', 2000), ('Texas', 2010)]]

state       year
California  2000    33871648
Texas       2010    25145561
dtype: int64

In [26]:
pop[[('California', 2010), ('New York', 2000)]]

state       year
California  2010    37253956
New York    2000    18976457
dtype: int64

In [27]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [28]:
pop.loc[('California', 2010):('New York', 2000)]

state       year
California  2010    37253956
New York    2000    18976457
dtype: int64

In [29]:
pop.iloc[1:3]

state       year
California  2010    37253956
New York    2000    18976457
dtype: int64

In [30]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [31]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [32]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

How about for a `DataFrame`?

In [33]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      19.0  37.6  58.0  38.1  52.0  36.5
     2      37.0  37.3  33.0  36.7  46.0  37.8
2014 1      35.0  37.4  30.0  36.2  39.0  38.2
     2      45.0  35.6  35.0  38.3  37.0  36.2

In [34]:
health_data['Guido', 'HR']

year  visit
2013  1        58.0
      2        33.0
2014  1        30.0
      2        35.0
Name: (Guido, HR), dtype: float64

In [35]:
health_data['Bob']

type          HR  Temp
year visit            
2013 1      19.0  37.6
     2      37.0  37.3
2014 1      35.0  37.4
     2      45.0  35.6

In [36]:
health_data.iloc[:3, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      19.0  37.6
     2      37.0  37.3
2014 1      35.0  37.4

In [37]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        19.0
      2        37.0
2014  1        35.0
      2        45.0
Name: (Bob, HR), dtype: float64

In [38]:
health_data.unstack()

subject   Bob                   Guido                     Sue              \
type       HR        Temp          HR        Temp          HR        Temp   
visit       1     2     1     2     1     2     1     2     1     2     1   
year                                                                        
2013     19.0  37.0  37.6  37.3  58.0  33.0  38.1  36.7  52.0  46.0  36.5   
2014     35.0  45.0  37.4  35.6  30.0  35.0  36.2  38.3  39.0  37.0  38.2   

subject        
type           
visit       2  
year           
2013     37.8  
2014     36.2

In [39]:
health_data.iloc[3,2:]

subject  type
Guido    HR      35.0
         Temp    38.3
Sue      HR      37.0
         Temp    36.2
Name: (2014, 2), dtype: float64

In [40]:
health_data.iloc[3,1]

35.6

## Rearranging Multi-Indices

One of the keys to working with multiply indexed data is knowing how to effectively transform the data.

Let's start by creating some simple multiply indexed data where the indices are not lexographically sorted:

In [41]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.250012
      2      0.063356
c     1      0.934155
      2      0.429901
b     1      0.785267
      2      0.743452
dtype: float64

In [42]:
data['a':'b']

UnsortedIndexError: 'Key length (1) was greater than MultiIndex lexsort depth (0)'

In [ ]:
# need to sort the index prior to slicing
data = data.sort_index()
data

char  int
a     1      0.314119
      2      0.988486
b     1      0.775234
      2      0.767484
c     1      0.051822
      2      0.828181
dtype: float64

In [ ]:
data['a':'b']

char  int
a     1      0.314119
      2      0.988486
b     1      0.775234
      2      0.767484
dtype: float64

## Stacking and unstacking indices
 
It is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representation, optionally specifying the level to use:

In [ ]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [ ]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [ ]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## Data Aggregations on Multi-Indices

We've previously seen that Pandas has built-in data aggregation methods, such as `mean()`, `sum()`, and `max()`. 

For hierarchically indexed data, these can be passed a level parameter that controls which subset of the data the aggregate is computed on:

In [ ]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      34.0  35.2  34.0  36.4  26.0  37.7
     2      54.0  37.0  43.0  37.4  41.0  36.2
2014 1      40.0  37.5  27.0  37.1  35.0  36.5
     2      37.0  38.1  60.0  36.7  53.0  37.2

In [ ]:
data_mean = health_data.groupby(level='year').mean()
data_mean

subject   Bob       Guido         Sue       
type       HR  Temp    HR  Temp    HR   Temp
year                                        
2013     44.0  36.1  38.5  36.9  33.5  36.95
2014     38.5  37.8  43.5  36.9  44.0  36.85

Can take the mean along levels of the columns too

In [ ]:
data_mean.groupby(axis=1, level='type').mean()

type,HR,Temp
year,,
2013,38.666667,36.650000
2014,42.000000,37.183333


In [ ]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      41.0  36.3  38.0  37.2  48.0  35.3
     2      36.0  36.0  25.0  38.1  36.0  39.3
2014 1      20.0  36.3  37.0  37.1  25.0  36.1
     2      25.0  36.8  33.0  38.9  35.0  37.5

In [ ]:
data_mean = health_data.groupby(level='year').mean()
data_mean

subject   Bob        Guido          Sue      
type       HR   Temp    HR   Temp    HR  Temp
year                                         
2013     38.5  36.15  31.5  37.65  42.0  37.3
2014     22.5  36.55  35.0  38.00  30.0  36.8

In [ ]:
data_max = health_data.groupby(level='year').max()
data_max

subject   Bob       Guido         Sue      
type       HR  Temp    HR  Temp    HR  Temp
year                                       
2013     41.0  36.3  38.0  38.1  48.0  39.3
2014     25.0  36.8  37.0  38.9  35.0  37.5

In [ ]:
data_mean2 = data_mean.groupby(axis=1, level='type').mean()
data_mean2

type,HR,Temp
year,,
2013,37.333333,37.033333
2014,29.166667,37.116667


# Summary

By allowing multi-indexing we enable a powerful querying mechanism over the data that we are using.

